In [ ]:
import pandas as pd

# Load the CSV files into DataFrames
appstore_df = pd.read_csv('./AppStoreOutput_cleaned.csv')
googleplay_df = pd.read_csv('./GooglePlayOutput_cleaned.csv')



print("App Store Data Info:")
print(appstore_df.info())

print("\nGoogle Play Store Data Info:")
print(googleplay_df.info())





In [9]:
import pandas as pd
from datetime import datetime, timezone
import re
from ast import literal_eval
from iso639 import languages
from collections import Counter
from transformers import pipeline
from transformers import AutoTokenizer
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from langdetect import detect
import os    


df = pd.read_csv('./AppStoreOutput.csv', delimiter=',', encoding='utf-8')
df['released'] = pd.to_datetime(df['released'])
df['updated'] = pd.to_datetime(df['updated'])
df['score'] = pd.to_numeric(df['score'], errors='coerce')
df['free'] = df['free'].astype(int)
df['supports_iPhone'] = 0
df['supports_iPad'] = 0
df['supports_Mac'] = 0
df['days_since_last_update'] = (datetime.now(timezone.utc) - df['updated']).dt.days
df['app_age'] = (df['updated'] - df['released']).dt.days
df['reviews'] = df['reviews'].astype(str)

# Load the sentiment analysis pipeline with the multilingual BERT model
sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


# Function to ensure stopwords are available
def ensure_stopwords():
# Define the default path (adjust as needed for your system)
    default_path = os.path.join(nltk.data.path[0], 'corpora', 'stopwords')
    if not os.path.exists(default_path):
        print("Downloading NLTK stopwords...")
        nltk.download('stopwords')
    else:
        print("Stopwords already installed.")

ensure_stopwords()


# Mapping from language names to NLTK compatible language codes
nltk_lang_map = {
    'ar': 'arabic',
    'az': 'azerbaijani',
    'eu': 'basque',
    'bn': 'bengali',
    'ca': 'catalan',
    'zh': 'chinese',
    'da': 'danish',
    'nl': 'dutch',
    'en': 'english',
    'fi': 'finnish',
    'fr': 'french',
    'de': 'german',
    'el': 'greek',
    'he': 'hebrew',
    'hu': 'hungarian',
    'id': 'indonesian',
    'it': 'italian',
    'kk': None,  # No support in NLTK
    'ne': None,  # No support in NLTK
    'no': 'norwegian',
    'pt': 'portuguese',
    'ro': 'romanian',
    'ru': 'russian',
    'sl': 'slovene',
    'es': 'spanish',
    'sv': 'swedish',
    'tg': None,  # No support in NLTK
    'tr': 'turkish'
}

def get_stopwords(text):
    try:
        # Detect the language of the text
        lang = detect(text)
        # Get the stopwords for the detected language
        stopwords_lang = nltk_lang_map.get(lang, 'english')
        if stopwords_lang:
            return set(stopwords.words(stopwords_lang))
        else:
            return set(stopwords.words('english'))
    except Exception as e:
        print("Error in detecting language or loading stopwords:", e)
        return set(stopwords.words('english'))

def preprocess_and_split_reviews(reviews):
    # Convert reviews to string to avoid TypeError with non-string inputs
    if pd.isna(reviews):
        return ""  # Return an empty string if the review is NaN
    reviews = str(reviews)
    try:
        # Use language-specific stopwords
        stop_words = get_stopwords(reviews)
    except Exception as e:
        print("Error using language-specific stopwords:", e)
        stop_words = set(stopwords.words('english'))  # Default to English if error occurs

    # Remove all non-alpha characters and extra spaces, convert to lower case
    reviews = re.sub(r'[^\\wáčďéěíňóřšťúůýžÁČĎÉĚÍŇÓŘŠŤÚŮÝŽ]', ' ', reviews, flags=re.UNICODE)
    reviews = re.sub('\\s+', ' ', reviews).strip().lower()

    # Remove stopwords
    words = [word for word in reviews.split() if word not in stop_words and len(word) > 1]
    return ' '.join(words)

# Apply the modified function to your DataFrame
df['processed_reviews'] = df['reviews'].apply(preprocess_and_split_reviews)

def get_and_flatten_bigrams(text):
    if len(text.split()) < 2:
        return []
    blob = TextBlob(text)
    bigrams = [' '.join(bigram) for bigram in blob.ngrams(2)]
    # Count bigrams
    bigram_counts = Counter(bigrams)
    # Return bigrams with their frequencies
    return list(bigram_counts.items())

# Apply the function
df['bigram_freq'] = df['processed_reviews'].apply(get_and_flatten_bigrams)
bigram_freq_rows = df.explode('bigram_freq')

# Create a new DataFrame that splits the bigram and frequency into separate columns
bigram_freq_df = pd.DataFrame({
    'appId': bigram_freq_rows['appId'],
    'bigrams': bigram_freq_rows['bigram_freq'].apply(lambda x: x[0] if pd.notna(x) else ''),
    'frequency': bigram_freq_rows['bigram_freq'].apply(lambda x: x[1] if pd.notna(x) else 0)
}).dropna()

def flatten_word_frequencies(text):
    words = text.split()
    freqs = Counter(words)
    # Get all word frequencies, without limiting to top 20
    return list(freqs.items())

# Apply the  function
df['word_freq'] = df['processed_reviews'].apply(flatten_word_frequencies)
word_freq_rows = df.explode('word_freq')
word_freq_df = pd.DataFrame({
    'appId': word_freq_rows['appId'],
    'word': word_freq_rows['word_freq'].apply(lambda x: x[0] if pd.notna(x) else ''),
    'frequency': word_freq_rows['word_freq'].apply(lambda x: x[1] if pd.notna(x) else 0)
}).dropna()


for index, row in df.iterrows():
    if 'iPhone' in row['supportedDevices']:
        df.at[index, 'supports_iPhone'] = 1
    if 'iPad' in row['supportedDevices']:
        df.at[index, 'supports_iPad'] = 1
    if 'Mac' in row['supportedDevices']:
        df.at[index, 'supports_Mac'] = 1



def compute_sentiment_category_mbert(text):
    try:
        # Directly pass the text to the pipeline
        # The pipeline handles tokenization and truncation internally
        result = sentiment_analyzer(text, truncation=True, max_length=512)[0]
        label = result['label']
        
        # Mapping the model output to custom categories
        if label == '1 star':
            return 'Negative'
        elif label == '2 stars':
            return 'Slightly negative'
        elif label == '3 stars':
            return 'Neutral'
        elif label == '4 stars':
            return 'Slightly positive'
        else:  # '5 stars'
            return 'Positive'
    except Exception as e:
        print(f"Error processing text: {e}")
        return 'Missing'  # Default to 'Missing' in case of an error

df['Sentiment_Category'] = df['processed_reviews'].apply(compute_sentiment_category_mbert)


# Parsing and One-hot Encoding for List Columns
def parse_list_column(column):
    try:
        return column.apply(literal_eval)
    except ValueError:
        return column

df['languages'] = parse_list_column(df['languages'])
df['genres'] = parse_list_column(df['genres'])

languages_exploded = df[['appId', 'languages']].explode('languages')
genres_exploded = df[['appId', 'genres']].explode('genres')


# Expanded Language-to-Countries Mapping
language_to_countries = {
    'AF': ['Afghanistan'],
    'AM': ['Armenia'],
    'AN': ['Netherlands Antilles'],
    'AR': ['Saudi Arabia', 'Iraq', 'Egypt', 'Algeria', 'Morocco', 'Sudan', 'Yemen', 'Syria', 'Tunisia', 'Jordan', 'Libya', 'Lebanon', 'Oman', 'Kuwait', 'Mauritania', 'Qatar', 'Bahrain', 'United Arab Emirates'],
    'AZ': ['Azerbaijan'],
    'BE': ['Belarus'],
    'BG': ['Bulgaria'],
    'BN': ['Bangladesh', 'India'],
    'BR': ['Brazil'],
    'BS': ['Bosnia and Herzegovina'],
    'CA': ['Spain', 'Andorra'],
    'CO': ['France'],
    'CS': ['Czech Republic'],
    'CY': ['Wales'],
    'DA': ['Denmark', 'Greenland', 'Faroe Islands'],
    'DE': ['Germany', 'Austria', 'Switzerland', 'Luxembourg', 'Liechtenstein'],
    'EL': ['Greece', 'Cyprus'],
    'EN': ['United States', 'United Kingdom', 'Canada', 'Australia', 'Ireland', 'New Zealand', 'South Africa'],
    'EO': ['Worldwide'],  # Esperanto is a constructed international auxiliary language.
    'ES': ['Spain', 'Mexico', 'Colombia', 'Argentina', 'Peru', 'Venezuela', 'Chile', 'Ecuador', 'Guatemala', 'Cuba', 'Bolivia', 'Dominican Republic', 'Honduras', 'Paraguay', 'El Salvador', 'Nicaragua', 'Costa Rica', 'Puerto Rico', 'Panama', 'Uruguay'],
    'ET': ['Estonia'],
    'EU': ['Spain'],  # Basque Country
    'FA': ['Iran', 'Afghanistan', 'Tajikistan'],
    'FI': ['Finland', 'Sweden'],
    'FR': ['France', 'Canada', 'Belgium', 'Switzerland', 'Luxembourg', 'Monaco', 'Congo', 'Ivory Coast', 'Madagascar', 'Cameroon', 'Burkina Faso', 'Niger', 'Senegal', 'Mali', 'Rwanda', 'Belgium', 'Guinea'],
    'FY': ['Netherlands'],
    'GA': ['Ireland'],
    'GD': ['Scotland'],
    'GL': ['Spain'],
    'GU': ['India'],
    'HE': ['Israel'],
    'HI': ['India'],
    'HR': ['Croatia', 'Bosnia and Herzegovina'],
    'HT': ['Haiti'],
    'HU': ['Hungary'],
    'HY': ['Armenia', 'Nagorno-Karabakh Republic'],
    'IA': ['Worldwide'],  # Interlingua is a constructed international auxiliary language.
    'ID': ['Indonesia'],
    'IG': ['Nigeria'],
    'IS': ['Iceland'],
    'IT': ['Italy', 'Switzerland', 'San Marino', 'Vatican City'],
    'JA': ['Japan'],
    'KA': ['Georgia'],
    'KK': ['Kazakhstan'],
    'KM': ['Cambodia'],
    'KN': ['India'],
    'KO': ['South Korea', 'North Korea'],
    'KU': ['Turkey', 'Iraq', 'Iran', 'Syria'],
    'KY': ['Kyrgyzstan'],
    'LO': ['Laos'],
    'LT': ['Lithuania'],
    'LV': ['Latvia'],
    'MK': ['North Macedonia'],
    'ML': ['India', 'Sri Lanka'],
    'MN': ['Mongolia'],
    'MR': ['India'],
    'MS': ['Malaysia', 'Brunei', 'Singapore'],
    'MT': ['Malta'],
    'MY': ['Myanmar'],
    'NB': ['Norway'],
    'NE': ['Niger'],
    'NL': ['Netherlands', 'Belgium', 'Suriname'],
    'NN': ['Norway'],
    'OC': ['France'],
    'PA': ['India', 'Pakistan'],
    'PL': ['Poland'],
    'PS': ['Afghanistan', 'Pakistan'],
    'PT': ['Portugal', 'Brazil', 'Angola', 'Mozambique', 'Cape Verde', 'Guinea-Bissau', 'São Tomé and Príncipe', 'East Timor'],
    'RO': ['Romania', 'Moldova'],
    'RU': ['Russia', 'Belarus', 'Kazakhstan', 'Kyrgyzstan'],
    'SC': ['Italy'],
    'SE': ['Sweden'],
    'SI': ['Sri Lanka'],
    'SK': ['Slovakia'],
    'SL': ['Slovenia'],
    'SN': ['Zimbabwe'],
    'SQ': ['Albania', 'Kosovo'],
    'SR': ['Serbia', 'Bosnia and Herzegovina', 'Montenegro', 'Kosovo'],
    'SV': ['Sweden'],
    'SW': ['Tanzania', 'Kenya', 'Uganda'],
    'TA': ['India', 'Sri Lanka'],
    'TE': ['India'],
    'TG': ['Tajikistan'],
    'TH': ['Thailand'],
    'TL': ['Timor-Leste'],
    'TR': ['Turkey', 'Cyprus'],
    'TT': ['Russia'],
    'UK': ['Ukraine'],
    'UR': ['Pakistan', 'India'],
    'UZ': ['Uzbekistan'],
    'VI': ['Vietnam'],
    'XH': ['South Africa'],
    'YI': ['Worldwide'],  # Yiddish is spoken by Jewish communities worldwide.
    'YO': ['Nigeria', 'Benin'],
    'ZH': ['China', 'Taiwan', 'Singapore', 'Malaysia'],
    'ZU': ['South Africa'],
}



# Map language codes to countries
languages_exploded['Countries'] = languages_exploded['languages'].map(lambda x: ', '.join(language_to_countries.get(x, ['Unknown'])))
languages_exploded['Countries'] = languages_exploded['Countries'].fillna('Unknown')
# Split the 'Countries' column into a list of countries
languages_exploded['Countries'] = languages_exploded['Countries'].str.split(', ')
# Explode the 'Countries' column
languages_exploded = languages_exploded.explode('Countries')

def get_language_name(lang_code):
    # Check if lang_code is not a string or if it's NaN
    if not isinstance(lang_code, str) or pd.isna(lang_code):
        return "en"  # Return 'Unknown' or some other placeholder

    # Convert the language code to lowercase to match the iso639 library's expected format
    lang_code_lower = lang_code.lower()
    try:
        # Attempt to get the language name using the ISO 639-1 code
        lang_name = languages.get(part1=lang_code_lower).name
    except KeyError:
        try:
            # If the ISO 639-1 code lookup fails, try the ISO 639-2/T code
            lang_name = languages.get(part2t=lang_code_lower).name
        except KeyError:
            try:
                # If the ISO 639-2/T code lookup also fails, try the ISO 639-2/B code
                lang_name = languages.get(part2b=lang_code_lower).name
            except KeyError:
                # If none of the lookups are successful, return the original code
                lang_name = lang_code  # Keeping the original case for visibility
    return lang_name

# Apply the function to translate language codes to names
languages_exploded['language_name'] = languages_exploded['languages'].apply(get_language_name)



# Categorizing Numerical Data
bins = [0, 50000000, 200000000, float('inf')]
labels = ['Small', 'Medium', 'Large']


# Function to convert binary values to 'free' or 'paid'
def free_convert_to_category(value):
    if value == 1:
        return 'free'
    else:
        return 'paid'

## App age categorization
def categorize_app_age(days):
    if days <= 30:
        return 'Brand New'
    elif days <= 90:
        return 'Recently Launched'
    elif days <= 365:
        return 'Established'
    elif days <= 1095:
        return 'Mature'
    else:
        return 'Very Mature'
    



# Define categories based on percentiles
def price_category(price):
    if price == 0:
        return 'Free'
    elif price <= percentilesPrice[0.25]:
        return 'Low'
    elif price <= percentilesPrice[0.50]:
        return 'Medium'
    elif price <= percentilesPrice[0.75]:
        return 'High'
    else:
        return 'Very High'    
        

## Update frequency categorization
def categorize_update_frequency(days_since_last_update):
    if days_since_last_update <= 30:
        return 'Very Recent Updates'
    elif days_since_last_update <= 90:
        return 'Recently Updated'
    elif days_since_last_update <= 180:
        return 'Moderately Updated'
    elif days_since_last_update <= 365:
        return 'Rarely Updated'
    else:
        return 'Stale'



# Process reviews
df['Sentiment_Category'] = df['reviews'].apply(compute_sentiment_category_mbert)

# Update frequency calculation
df['update_frequency'] = df['days_since_last_update'].apply(categorize_update_frequency)

## Apply the categorization function to the 'price' column
percentilesPrice = df['price'].quantile([0.25, 0.50, 0.75])
df['price_category'] = df['price'].apply(price_category)

## Apply the categorization function to the 'app_age' column
df['app_age_category'] = df['app_age'].apply(lambda days: categorize_app_age(days))

## Apply the categorization function to the 'file_size' column
df['file_size_category'] = pd.cut(df['size'], bins=bins, labels=labels)


# Aggregate Languages and Genres
language_counts = df.filter(regex='^lang_').sum().reset_index()
genre_counts = df.filter(regex='^_').sum().reset_index()

# Remove '_' prefix and rename columns
genre_counts['index'] = genre_counts['index'].str.replace('^_', '', regex=True)
genre_counts.columns = ['Genre', 'Number of Apps']

# Remove 'lang_' prefix and rename columns
language_counts['index'] = language_counts['index'].str.replace('lang_', '', regex=True)
language_counts.columns = ['Language', 'Number of Apps']


# Creating a relational table for device support
device_support = df.melt(id_vars=['appId'], value_vars=['supports_iPhone', 'supports_iPad', 'supports_Mac'], var_name='Device', value_name='Supported')
device_support = device_support[device_support['Supported'] == 1].drop('Supported', axis=1)
device_support.to_csv('AppStore_Device_Support.csv', index=False)


# Apply the function to the 'free' column
df['free'] = df['free'].apply(free_convert_to_category)


# Rename columns for clarity
language_counts.columns = ['Language', 'Number of Apps']
genre_counts.columns = ['Genre', 'Number of Apps']

# Preview the aggregated language data
print(language_counts.head())
print('\n')
# Preview the aggregated genre data
print(genre_counts.head())

# Device support aggregation
device_support_counts = df[['supports_iPhone', 'supports_iPad', 'supports_Mac']].sum().reset_index()

# Rename columns for clarity
device_support_counts.columns = ['Device Type', 'Number of Apps']

# Convert device type names to more readable format if necessary
# Example: You can manually rename each type for clarity
device_support_counts['Device Type'] = device_support_counts['Device Type'].replace({
    'supports_iPhone': 'iPhone',
    'supports_iPad': 'iPad',
    'supports_Mac': 'Mac'
})

# Preview the device support data
print(device_support_counts)




# Final DataFrame Cleanup and Saving the Cleaned Data
columns_to_remove = [
    'id', '', 'description', 'icon', 'genreIds', 'primaryGenreId',
    'requiredOsVersion', 'releaseNotes', 'version', 'developerid', 'developerUrl',
    'screenshots', 'ipadScreenshots', 'appletvScreenshots',
    'languages', 'genres', 'supportedDevices', 'currency', 'developerId', 'reviews', 
]
df.drop(columns_to_remove, axis=1, inplace=True, errors='ignore')

# Save the modified DataFrame to a new CSV file
df.to_csv('AppStoreOutput_cleaned.csv', index=False, sep=',', encoding='utf-8')
languages_exploded.to_csv('AppStore_Languages.csv', index=False)
genres_exploded.to_csv('AppStore_Genres.csv', index=False)

# Save the results to separate CSV files
bigram_freq_df.to_csv('AppStore_Bigrams.csv', index=False)
word_freq_df.to_csv('AppStore_Word_Frequencies.csv', index=False)
print("Bigrams and word frequencies have been saved to CSV files.")

# Preview the DataFrame
print(df.head())

Stopwords already installed.
Empty DataFrame
Columns: [Language, Number of Apps]
Index: []


Empty DataFrame
Columns: [Genre, Number of Apps]
Index: []
  Device Type  Number of Apps
0      iPhone              20
1        iPad              20
2         Mac               1
Bigrams and word frequencies have been saved to CSV files.
                              appId                           title  \
0                 com.wanderlog.ios      Wanderlog - Travel Planner   
1           com.tripit.tripitmobile          TripIt: Travel Planner   
2                    app.tripsy.ios  Tripsy: Travel Planner & Guide   
3  com.roadtrippers.roadtrippersinc     Roadtrippers - Trip Planner   
4        com.tripadvisor.LocalPicks  Tripadvisor: Plan & Book Trips   

                                                 url primaryGenre  \
0  https://apps.apple.com/us/app/wanderlog-travel...       Travel   
1  https://apps.apple.com/us/app/tripit-travel-pl...       Travel   
2  https://apps.apple.com/us/app/tr

In [ ]:
from nltk.corpus import stopwords
stopwords.fileids()